## Check if Gaia resolves any of our TESS targets into more than one GAIA object
(From 2024-01-07 Meeting)

In [2]:
from sqlite3 import connect
from astropy.coordinates import SkyCoord
from astropy.table import Table, vstack
from astropy.time import Time
import astropy.units as u
from astroquery.gaia import Gaia
from astroquery.mast import Catalogs
import numpy as np
import pandas as pd

In [3]:
conn = connect("../django/QuadEBdb/db.sqlite3")

# read base list of targets
targets = pd.read_sql("select * from tom_target;", conn, index_col="id")

# retrieve all objects by TIC ID from the TIC catalog, which (among other things) gives us the GAIA ID
# TIC fields: https://mast.stsci.edu/api/v0/_t_i_cfields.html
tic_table = Catalogs.query_criteria(
    catalog="Tic", ID=targets["local_id"].str.replace("TIC ", "")
)

# replace NaN values with a stand-ins
tic_table["pmRA"][np.isnan(tic_table["pmRA"])] = 0
tic_table["pmDEC"][np.isnan(tic_table["pmDEC"])] = 0
tic_table["plx"][np.isnan(tic_table["plx"])] = 1e-3

# write to a csv file in case we want to do other analysis
# tic_table.write("TICv8 Targets.csv", format="ascii.csv", overwrite=True)

In [5]:
tic_epoch = Time(2000.0, format="jyear", scale="tdb")
gaia_dr3_epoch = Time(2016.0, format="jyear", scale="tdb")
cone_size = 10 * u.arcsec

gaia_neighbors = None
gaia_from_tic = {}
for tic_object in tic_table:
    tic_coord = SkyCoord(
        ra=tic_object["ra"] * u.deg,
        dec=tic_object["dec"] * u.deg,
        distance=tic_object["plx"] * u.mas,
        pm_ra_cosdec=tic_object["pmRA"] * u.mas / u.yr,
        pm_dec=tic_object["pmDEC"] * u.mas / u.yr,
        obstime=tic_epoch,
    )
    # evolve coord to gaia epoch
    tic_coord = tic_coord.apply_space_motion(gaia_dr3_epoch)

    # GAIA fields: https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html
    job = Gaia.launch_job(
        "select * "
        "from gaiadr3.gaia_source "
        f"WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', {tic_coord.ra.to(u.deg).value}, {tic_coord.dec.to(u.deg).value}, {cone_size.to(u.deg).value})) = 1"
    )
    gaia_matches = job.get_results()

    #add reference to the TIC ID for these matches
    gaia_matches["TIC ID"] = tic_object["ID"]

    #calculate distance between each match and the TIC coordinates
    distances = [
        tic_coord.separation(SkyCoord(
            ra=gaia_object["ra"] * u.deg,
            dec=gaia_object["dec"] * u.deg,
            distance=gaia_object["parallax"] * u.mas,
        ))
        for gaia_object in gaia_matches]
    gaia_matches["TIC Distance"] = distances

    # store results for later use
    gaia_from_tic[tic_object["ID"]] = (tic_coord, gaia_matches)

    # build a table of all matches
    if gaia_neighbors is None:
        gaia_neighbors = gaia_matches.copy()
    else:
        gaia_neighbors = vstack([gaia_neighbors, gaia_matches])
    print(f"{tic_object["ID"]} ({len(gaia_neighbors)})", end=" ")

# gaia_neighbors.write("GAIA Neighbors.csv", format="ascii.csv", overwrite=True)


74919381 (3) 

c:\Users\User\Anaconda3\envs\Astro\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 1 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


140593601 (6) 

c:\Users\User\Anaconda3\envs\Astro\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 1 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


75740921 (9) 

...

278465736 (572) 127011022 (574) 285853156 (576) 274862252 (581) 

c:\Users\User\Anaconda3\envs\Astro\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 1 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


278142612 (582) 

c:\Users\User\Anaconda3\envs\Astro\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 1 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


336882813 (583) 309262405 (585) 

In [38]:
separation_threshold = 2 * u.arcsec

match_results = {}
with open("..\\django\\QuadEBdb\\Data Files\\Catalog Associations\\Gaia objects close to TESS objects.csv", "w") as f:
    f.write("Local ID,Catalog,Catalog ID,Association\n")
    for tic_id, (tic_coord, gaia_matches) in gaia_from_tic.items():
        distances = []
        for gaia_object in gaia_matches:
            gaia_coord = SkyCoord(
                ra=gaia_object["ra"] * u.deg,
                dec=gaia_object["dec"] * u.deg,
                distance=gaia_object["parallax"] * u.mas,
            )
            distance_to_tic = tic_coord.separation(gaia_coord)
            # if distance_to_tic < separation_threshold:
            #     f.write(f"TIC {tic_id},GAIA DR3,{gaia_object["source_id"]},GAIA object within {separation_threshold} of TIC object\n")
            distances.append((distance_to_tic, gaia_object["DESIGNATION"], gaia_object["parallax"], gaia_object["phot_g_mean_mag"], gaia_object["pmra"], gaia_object["pmdec"], gaia_object["source_id"]))

    close_enough = sorted(filter(lambda dist: dist[0] < separation_threshold, distances))
    too_far = sorted(filter(lambda dist: dist[0] > separation_threshold, distances))
    if len(close_enough) > 1:
        print(f"TIC {tic_id}")
        for dist in close_enough:
            print(f"    {dist[1]:<30s} {dist[0].to(u.arcsec):.3f} {dist[2]:6.4f} {dist[3]:.2f} {dist[4]:.3f} {dist[5]:.3f}")
        if len(too_far) > 0:
            print("    ----------------------------")
            for dist in too_far:
                print(f"    {dist[1]:<30s} {dist[0].to(u.arcsec):.3f}")
        match_results[tic_id] = (len(close_enough), len(too_far))

In [25]:
gaia_neighbors

solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,TIC ID,TIC Distance
,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / um,1 / um,1 / um,,,,,,,,mas,,,,,deg,,,,,,,,deg,deg,deg,deg,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,,,,,mag,mag,mag,km / s,km / s,,,,,,,,d,km / s,K,log(cm.s**-2),dex,,km / s,km / s,,mag,mag,,,,deg,deg,deg,deg,,,,,,,,,,,,,,,K,K,K,log(cm.s**-2),log(cm.s**-2),log(cm.s**-2),dex,dex,dex,pc,pc,pc,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,deg
int64,object,int64,int64,float64,float64,float32,float64,float32,float64,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,int16,bool,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32,int16,int16,int16,float32,float32,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,float32,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,float32